In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch
import torch.nn as nn
import torch.nn.functional as F 

from safetensors import torch as sftorch

from huggingface_hub import hf_hub_download

import sys
sys.path.append("./llama_architecture")

from model_trnsfmrs import LlamaForCausalLM
from config import LlamaConfig


In [2]:
device = "mps"

In [3]:
def count_parameters(model):
    """Count the number of parameters in a model"""
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [4]:
tokenizer = AutoTokenizer.from_pretrained("aliarda/turkish-news-32k-tokenizer")
model_path = hf_hub_download(repo_id="aliarda/llama-50M-latest", filename="model.safetensors")
state_dict = sftorch.load_file(model_path, device=device)


In [8]:
llama_config = LlamaConfig(
    vocab_size=32768,
    emb_dim=256,
    context_length=256,
    n_heads=128,
    n_layers=20,
    n_kv_groups=64,
    hidden_dim=2048,
)

llama_model = LlamaForCausalLM(llama_config, tokenizer)
print(count_parameters(llama_model))
llama_model

52177152


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32768, 256)
    (layers): ModuleList(
      (0-19): 20 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=128, bias=False)
          (v_proj): Linear(in_features=256, out_features=128, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=256, out_features=2048, bias=False)
          (up_proj): Linear(in_features=256, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=256, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=256, out_features=32768, bias=False)
)

In [6]:
llama_model.load_state_dict(state_dict)
llama_model.to(device)
llama_model.device = device
llama_model.context_len = 256

In [7]:
llama_model.generate("merhaba sayın okuyucular")

256


'merhaba sayın okuyucularım, siz bu yazıyı yazdım ve yazdım.\n[old_news_related_template title="Türkiye\'de koronavirüs vakaları artmayadevamediyor"desc="Türkiye\'de koronavirüs vakalarının artmasıüzerine SağlıkBakanı Fahrettin Koca\'nın açıkladığı koronavirüs vakasayısı merakediliyor.Pekiama bugünkü vakasayısı kaç oldu? 18 Temmuz 2021 bugünkü vakasayısı kaç oldu? 18 Temmuz 2021 koronavirüs tablosu\nSondakika gündem haberinegöre SağlıkBakanı Fahrettin Koca\'nın açıkladığı koronavirüs tablosu merakediliyor.Pekiama bugünkü vakasayısı kaç oldu? 18 Temmuz 2021 koronavirüs tablosu merakediliyor.Sağlık Bakanı Fahrettin Koca koronavirüsle mücadelede sondurum ne? sorusunun yanıtını araştırıyor.Pekiama bugünkü vakasayısı kaç oldu? 20 Ocak 2021 koronavirüs tablosu…\nİLTİFEKSİYONLARINDA SON DURUM NE: Koronavirüs vakasayısı ve tablosu merakediliyor.Sağlık Bakanı Fahrettin Koca koronavirüsle mücadelede alınan önlemler konusunda vatandaşlara uyarılarda bulundu.Pekiama bugünkü vakasayısı kaç oldu? 20